## Libraries

In [29]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate, Activation, Concatenate
from tensorflow.keras.metrics import IoU, BinaryIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
import cv2 as cv
import datetime
import matplotlib.pyplot as plt

## Loading Data

In [2]:
home = os.environ['HOME']

In [3]:
path_X = os.path.join(home,'raw_data/image_slices')
path_y = os.path.join(home,'raw_data/mask_slices')

In [4]:
# path_small_X = os.path.join(home,'raw_data/small_dataset/sample_images')
# path_small_y = os.path.join(home,'raw_data/small_dataset/sample_masks')

In [5]:
split_ratio = 0.9

In [6]:
def train_val_split (path_X, path_y, split_ratio):
    X_names = os.listdir(path_X)
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    X_path = [f'{path_X}/{file}' for file in X_names]
    train_X, val_X = X_path[:int(len(X_path)*split_ratio)], X_path[int(len(X_path)*split_ratio):]
    train_y, val_y = y_path[:int(len(y_path)*split_ratio)], y_path[int(len(y_path)*split_ratio):]
    return train_X, val_X, train_y, val_y 

In [7]:
# train_X, val_X, train_y, val_y = train_val_split (path_small_X, path_small_y, split_ratio) #small dataset

In [8]:
train_X, val_X, train_y, val_y = train_val_split (path_X, path_y, split_ratio)

In [9]:
def verify_matching_input_labels(X_names, y_names):
    for x, y in zip(X_names, y_names):
        if os.path.basename(x) != os.path.basename(y):
            raise ValueError(f"X and Y not matching: {x, y}")

In [10]:
verify_matching_input_labels(train_X, train_y)

In [11]:
verify_matching_input_labels(val_X, val_y)

In [12]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1) / 255 
    return image, mask

In [13]:
def batch_data (X_path, y_path, batch_size):
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.shuffle(buffer_size = len(X_path), seed = 10).map(process_path).batch(batch_size)

### Training Dataset

In [14]:
train_dataset = batch_data(train_X, train_y, batch_size=16)

2022-12-04 16:38:30.905984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 16:38:30.917380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 16:38:30.919191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 16:38:30.922084: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

### Validation Dataset

In [15]:
val_dataset = batch_data(val_X, val_y, batch_size=16)

### Test Dataset

In [16]:
path_X_TEST = os.path.join(home,'raw_data/TEST_slices/test_image_slices')
path_y_TEST = os.path.join(home,'raw_data/TEST_slices/test_mask_slices')

In [17]:
def batch_data_test (path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size)

In [18]:
TEST_dataset = batch_data_test(path_X_TEST, path_y_TEST, batch_size=16)

## Model Definition

### ResNet

In [19]:
def batchnorm_relu(inputs):
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

In [20]:
def residual_block(inputs, num_filters, strides=1):
    #Convolutional Layer
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)
    
    #Shortcut
    shortcut = Conv2D(num_filters, (1,1), padding='same', strides=strides)(inputs)
               
    #Addition ofthe convolutional layer and shortcut
    x = x + shortcut
    
    return x

**UpSampling2D** is just a *simple scaling* up of the image by using nearest neighbour or bilinear upsampling, so nothing smart. Advantage is it's cheap.

**Conv2DTranspose** is a *convolution operation whose kernel is learnt* (just like normal conv2d operation) while training your model. Using Conv2DTranspose will also upsample its input but the key difference is the model should learn what is the best upsampling for the job.

In [21]:
def decoder_block(inputs, skip_features, num_filters):
    # = UpSampling2D((2, 2))(inputs)
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = residual_block(x, num_filters, strides=1)
    return x

In [22]:
def dice_loss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(targets * inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

In [23]:
def loss_sum(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    o = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred) + dice_loss(y_true, y_pred)
    return tf.reduce_mean(o)

In [24]:
def build_resunet(img_height, img_width, channels):
    
    #Inputs
    inputs = Input((img_height, img_width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255
    
    #Encoder 1
    x = Conv2D(64, (3,3), padding='same', strides=1)(inputs)
    x = batchnorm_relu(x)
    x = Conv2D(64, (3,3), padding='same', strides=1)(x)
    shortcut = Conv2D(64, (1,1), padding='same', strides=1)(inputs) #shortcut using the identity matrix
    skip1 = x + shortcut #this referes to the skip connection for the decoder
    
    #Encoder 2 and 3
    skip2 = residual_block(skip1, 128, strides=2)
    skip3 = residual_block(skip2, 256, strides=2)
    
    #Bridge/Bottleneck
    b = residual_block(skip3, 512, strides=2)
    
    #Decoder 1, 2, 3
    x = decoder_block(b, skip3, 256)
    x = decoder_block(x, skip2, 128)
    x = decoder_block(x, skip1, 64)
    
    #Classifier
    outputs= Conv2D(1, (1,1), padding='same', activation='sigmoid')(x)
    
    #Model
    model = Model(inputs, outputs)
    
    #Metrics
    iou = BinaryIoU()
    
    #Compile
    model.compile(optimizer='adam', loss=loss_sum, metrics=['accuracy', iou])
    
    model.summary()
       
    return model 

In [25]:
model = build_resunet(256, 256, 3)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[1][0]']                 
 alization)                     )                                                             

In [30]:
checkpoint_filepath = '../tmp/resnet/version1'
es = EarlyStopping(patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', restore_best_weights=True)

#log data for tensorboard visualization
logs_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = logs_dir , histogram_freq=1)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs = 100, callbacks=[es, checkpoint, tb_callback], verbose=1)

Epoch 1/100


2022-12-04 16:41:41.714526: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


3845/3845 [==============================] - 3747s 975ms/step - loss: 0.4900 - accuracy: 0.9282 - binary_io_u: 0.7735 - val_loss: 0.5352 - val_accuracy: 0.9236 - val_binary_io_u: 0.7501
Epoch 3/100
3845/3845 [==============================] - 3730s 970ms/step - loss: 0.2575 - accuracy: 0.9622 - binary_io_u: 0.8726 - val_loss: 0.3143 - val_accuracy: 0.9539 - val_binary_io_u: 0.8457
Epoch 14/100
3845/3845 [==============================] - 3723s 968ms/step - loss: 0.2511 - accuracy: 0.9632 - binary_io_u: 0.8757 - val_loss: 0.3103 - val_accuracy: 0.9539 - val_binary_io_u: 0.8484
Epoch 15/100
3845/3845 [==============================] - 3731s 970ms/step - loss: 0.2426 - accuracy: 0.9645 - binary_io_u: 0.8799 - val_loss: 0.3084 - val_accuracy: 0.9549 - val_binary_io_u: 0.8488
Epoch 16/100
3845/3845 [==============================] - 3729s 970ms/step - loss: 0.2356 - accuracy: 0.9655 - binary_io_u: 0.8829 - val_loss: 0.3094 - val_accuracy: 0.9551 - val_binary_io_u: 0.8473
Epoch 17/100
3222/3

In [33]:
print("Number of epochs:", len(history.history['val_binary_io_u'])) 

Number of epochs: 31


In [34]:
print("Validation loss:", round(np.min(history.history['val_loss']),4))

Validation loss: 0.2913


In [35]:
print("Validation accuracy:", round(np.max(history.history['val_accuracy']),4))

Validation accuracy: 0.9573


In [36]:
print("Validation iou:", round(np.max(history.history['val_binary_io_u']),4))

Validation iou: 0.8564


In [37]:
model.evaluate(TEST_dataset)

565/565 [==============================] - 162s 286ms/step - loss: 0.3048 - accuracy: 0.9574 - binary_io_u: 0.8477


[0.3048243820667267, 0.9573831558227539, 0.8477343916893005]

In [38]:
model.save_weights('../tmp/resnet/version1')